# Day 0

Import commonly used modules.

In [1]:
using DelimitedFiles
using Base.Iterators
using Combinatorics

# Day 1

## Part 1

After feeling like you've been falling for a few minutes, you look at the device's tiny screen. "Error: Device must be calibrated before first use. Frequency drift detected. Cannot maintain destination lock." Below the message, the device shows a sequence of changes in frequency (your puzzle input). A value like +6 means the current frequency increases by 6; a value like -3 means the current frequency decreases by 3.

For example, if the device displays frequency changes of +1, -2, +3, +1, then starting from a frequency of zero, the following changes would occur:

Current frequency  0, change of +1; resulting frequency  1.
Current frequency  1, change of -2; resulting frequency -1.
Current frequency -1, change of +3; resulting frequency  2.
Current frequency  2, change of +1; resulting frequency  3.
In this example, the resulting frequency is 3.

Here are other example situations:

+1, +1, +1 results in  3
+1, +1, -2 results in  0
-1, -2, -3 results in -6
Starting with a frequency of zero, what is the resulting frequency after all of the changes in frequency have been applied?

Inputs saved in day1/input1.txt

In [2]:
open("day1input.txt") do f
    global d1inputs = readdlm(f, Int64)
end

pos = sum(d1inputs)
print("Final resulting frequency: $pos")

Final resulting frequency: 595

## Part Two

You notice that the device repeats the same frequency change list over and over. To calibrate the device, you need to find the first frequency it reaches twice.

For example, using the same list of changes above, the device would loop as follows:

Current frequency  0, change of +1; resulting frequency  1.
Current frequency  1, change of -2; resulting frequency -1.
Current frequency -1, change of +3; resulting frequency  2.
Current frequency  2, change of +1; resulting frequency  3.
(At this point, the device continues from the start of the list.)
Current frequency  3, change of +1; resulting frequency  4.
Current frequency  4, change of -2; resulting frequency  2, which has already been seen.
In this example, the first frequency reached twice is 2. Note that your device might need to repeat its list of frequency changes many times before a duplicate frequency is found, and that duplicates might be found while in the middle of processing the list.

Here are other examples:

+1, -1 first reaches 0 twice.
+3, +3, +4, -2, -4 first reaches 10 twice.
-6, +3, +8, +5, -6 first reaches 5 twice.
+7, +7, -2, -7, -4 first reaches 14 twice.

What is the first frequency your device reaches twice?

In [3]:
using Base.Iterators

visited = Set{Int64}()
pos = 0
push!(visited, pos)

for input in cycle(d1inputs)
    pos += input
    if in(pos, visited)
        break
    end
    push!(visited, pos)
end

print("First frequency reached twice is: $pos")

First frequency reached twice is: 80598

# Day 2: Inventory Management System

You stop falling through time, catch your breath, and check the screen on the device. "Destination reached. Current Year: 1518. Current Location: North Pole Utility Closet 83N10." You made it! Now, to find those anomalies.

Outside the utility closet, you hear footsteps and a voice. "...I'm not sure either. But now that so many people have chimneys, maybe he could sneak in that way?" Another voice responds, "Actually, we've been working on a new kind of suit that would let him fit through tight spaces like that. But, I heard that a few days ago, they lost the prototype fabric, the design plans, everything! Nobody on the team can even seem to remember important details of the project!"

"Wouldn't they have had enough fabric to fill several boxes in the warehouse? They'd be stored together, so the box IDs should be similar. Too bad it would take forever to search the warehouse for two similar box IDs..." They walk too far away to hear any more.

Late at night, you sneak to the warehouse - who knows what kinds of paradoxes you could cause if you were discovered - and use your fancy wrist device to quickly scan every box and produce a list of the likely candidates (your puzzle input).

To make sure you didn't miss any, you scan the likely candidate boxes again, counting the number that have an ID containing exactly two of any letter and then separately counting those with exactly three of any letter. You can multiply those two counts together to get a rudimentary checksum and compare it to what your device predicts.

For example, if you see the following box IDs:

- `abcdef` contains no letters that appear exactly two or three times.
- `bababc` contains two a and three b, so it counts for both.
- `abbcde` contains two b, but no letter appears exactly three times.
- `abcccd` contains three c, but no letter appears exactly two times.
- `aabcdd` contains two a and two d, but it only counts once.
- `abcdee` contains two e.
- `ababab` contains three a and three b, but it only counts once.

Of these box IDs, four of them contain a letter which appears exactly twice, and three of them contain a letter which appears exactly three times. Multiplying these together produces a checksum of 4 * 3 = 12.

What is the checksum for your list of box IDs?

In [4]:
function getLetterCounts(s)
    counts = Dict()
    for c in s
        counts[c] = get(counts, c, 0) + 1
    end
    return counts
end

# Example function input (remove ';' to show output)
getLetterCounts("bababc");

In [5]:
function numCodesWithNRepeats(n, inputs)
    letters = Char[]
    num = 0
    for code in inputs
        counts = getLetterCounts(code)
        if n in values(counts)
            num +=1
        end
    end
    return num
end

# Example function input (remove ';' to show output)
numCodesWithNRepeats(2, ["abbcde", "bababc", "abcdef"]);

In [6]:
open("day2input.txt") do f
    global d2inputs = readdlm(f, String)
end

function checksum(inputs)
    repeat2 = numCodesWithNRepeats(2, inputs)
    repeat3 = numCodesWithNRepeats(3, inputs)
    return repeat2 * repeat3
end

checksum(d2inputs)

6370

## Part Two

Confident that your list of box IDs is complete, you're ready to find the boxes full of prototype fabric.

The boxes will have IDs which differ by exactly one character at the same position in both strings. For example, given the following box IDs:

abcde
fghij
klmno
pqrst
fguij
axcye
wvxyz
The IDs abcde and axcye are close, but they differ by two characters (the second and fourth). However, the IDs fghij and fguij differ by exactly one character, the third (h and u). Those must be the correct boxes.

What letters are common between the two correct box IDs? (In the example above, this is found by removing the differing character from either ID, producing fgij.)

In [7]:
function numChanges(a, b)
    n = 0
    for (c1, c2) in zip(a, b)
        if c1 != c2
            n +=1
        end
    end
    return n
end

numChanges("fghij", "fguij");

In [8]:
function pairsWithOneChange(inputs)
    pairs = []
    for (s1, s2) in combinations(inputs, 2)
        if numChanges(s1, s2) == 1
            push!(pairs, (s1, s2))
        end
    end
    return pairs
end

pairsWithOneChange(["abc", "acd", "adc", "aec"]);

In [9]:
function commonLetters(s1, s2)
    output = []
    for (c1, c2) in zip(s1, s2)
        if c1 == c2
            push!(output, c1)
        end
    end
    return join(output)
end

commonLetters("123456789", "123356789");

In [10]:
function commonOfCorrect(inputs)
    oneChange = pairsWithOneChange(inputs)
    commonLetters(oneChange[1]...)
end

commonOfCorrect(d2inputs)

"rmyxgdlihczskunpfijqcebtv"

# Day 3: No Matter How You Slice It

The Elves managed to locate the chimney-squeeze prototype fabric for Santa's suit (thanks to someone who helpfully wrote its box IDs on the wall of the warehouse in the middle of the night). Unfortunately, anomalies are still affecting them - nobody can even agree on how to cut the fabric.

The whole piece of fabric they're working on is a very large square - at least 1000 inches on each side.

Each Elf has made a claim about which area of fabric would be ideal for Santa's suit. All claims have an ID and consist of a single rectangle with edges parallel to the edges of the fabric. Each claim's rectangle is defined as follows:

The number of inches between the left edge of the fabric and the left edge of the rectangle.
The number of inches between the top edge of the fabric and the top edge of the rectangle.
The width of the rectangle in inches.
The height of the rectangle in inches.
A claim like `#123 @ 3,2: 5x4` means that claim ID 123 specifies a rectangle 3 inches from the left edge, 2 inches from the top edge, 5 inches wide, and 4 inches tall. Visually, it claims the square inches of fabric represented by # (and ignores the square inches of fabric represented by .) in the diagram below:

```...........
...........
...#####...
...#####...
...#####...
...#####...
...........
...........
...........```

The problem is that many of the claims overlap, causing two or more claims to cover part of the same areas. For example, consider the following claims:

`#1 @ 1,3: 4x4`

`#2 @ 3,1: 4x4`

`#3 @ 5,5: 2x2`

Visually, these claim the following areas:

```........
...2222.
...2222.
.11XX22.
.11XX22.
.111133.
.111133.
........```

The four square inches marked with X are claimed by both 1 and 2. (Claim 3, while adjacent to the others, does not overlap either of them.)

If the Elves all proceed with their own plans, none of them will have enough fabric. How many square inches of fabric are within two or more claims?

In [11]:
function parseClaim(s)
    split(s, r"[^0-9]") |>
    parts -> filter(!isempty, parts) |>
    nums -> map(n -> parse(Int64, n), nums)
end

parseClaim (generic function with 1 method)

In [12]:
function getAffectedSquaresIterator(_, x, y, w, h)
    Iterators.product(x:x+w-1, y:y+h-1)
end
getAffectedSquares(parseClaim("#1 @ 1,3: 4x4")...)

UndefVarError: UndefVarError: getAffectedSquares not defined

In [13]:
function getInput()
    open("day3input.txt") do f
        readlines(f) |>
        collect |>
        lines -> map(line -> parseClaim(line), lines)
    end
end;

In [14]:
function addPointToDict!(dict, xy)
    dict[(xy[1], xy[2])] = get(dict, (xy[1], xy[2]), 0) + 1
end

function addClaimToDict!(dict, claim)
    getAffectedSquaresIterator(claim...) |>
    iter -> foreach(xy -> addPointToDict!(dict, xy), iter)
end
dict = Dict()
addClaimToDict!(dict, (1, 1, 3, 4, 4))
dict

Dict{Any,Any} with 16 entries:
  (4, 3) => 1
  (1, 6) => 1
  (3, 6) => 1
  (2, 3) => 1
  (2, 6) => 1
  (4, 4) => 1
  (4, 6) => 1
  (2, 5) => 1
  (1, 4) => 1
  (3, 3) => 1
  (1, 3) => 1
  (4, 5) => 1
  (2, 4) => 1
  (1, 5) => 1
  (3, 5) => 1
  (3, 4) => 1

In [15]:
function part1()
    claims = getInput()
    coverage = Dict()
    for claim in claims
        addClaimToDict!(coverage, claim)
    end
    overlaps = 0
    for (k, v) in coverage
        if v > 1
            overlaps += 1
        end
    end
    return overlaps
end
print("There are " * string(part1()) * " square inches of overlap.")

There are 101469 square inches of overlap.

In [16]:
function noOverlappingSquares(dict, claim)
    affectedSquares = getAffectedSquares(claim...)
    for square in affectedSquares
        if dict[square] > 1
            return false
        end
    end
    return true
end
        
function part2()
    claims = getInput()
    overlappingSquareDict = Dict()
    for claim in claims
        addClaimToDict!(overlappingSquareDict, claim)
    end
    
    filter(claim -> noOverlappingSquares(overlappingSquareDict, claim), claims)[1][1]
end
print("Claim #" * string(part2()) * " has no overlapping squares.")

UndefVarError: UndefVarError: getAffectedSquares not defined

# Day 4: Repose Record

You've sneaked into another supply closet - this time, it's across from the prototype suit manufacturing lab. You need to sneak inside and fix the issues with the suit, but there's a guard stationed outside the lab, so this is as close as you can safely get.

As you search the closet for anything that might help, you discover that you're not the first person to want to sneak in. Covering the walls, someone has spent an hour starting every midnight for the past few months secretly observing this guard post! They've been writing down the ID of the one guard on duty that night - the Elves seem to have decided that one guard was enough for the overnight shift - as well as when they fall asleep or wake up while at their post (your puzzle input).

For example, consider the following records, which have already been organized into chronological order:

```
[1518-11-01 00:00] Guard #10 begins shift
[1518-11-01 00:05] falls asleep
[1518-11-01 00:25] wakes up
[1518-11-01 00:30] falls asleep
[1518-11-01 00:55] wakes up
[1518-11-01 23:58] Guard #99 begins shift
[1518-11-02 00:40] falls asleep
[1518-11-02 00:50] wakes up
[1518-11-03 00:05] Guard #10 begins shift
[1518-11-03 00:24] falls asleep
[1518-11-03 00:29] wakes up
[1518-11-04 00:02] Guard #99 begins shift
[1518-11-04 00:36] falls asleep
[1518-11-04 00:46] wakes up
[1518-11-05 00:03] Guard #99 begins shift
[1518-11-05 00:45] falls asleep
[1518-11-05 00:55] wakes up
```

Timestamps are written using year-month-day hour:minute format. The guard falling asleep or waking up is always the one whose shift most recently started. Because all asleep/awake times are during the midnight hour (00:00 - 00:59), only the minute portion (00 - 59) is relevant for those events.

Visually, these records show that the guards are asleep at these times:

```
Date   ID   Minute
            000000000011111111112222222222333333333344444444445555555555
            012345678901234567890123456789012345678901234567890123456789
11-01  #10  .....####################.....#########################.....
11-02  #99  ........................................##########..........
11-03  #10  ........................#####...............................
11-04  #99  ....................................##########..............
11-05  #99  .............................................##########.....
```

The columns are Date, which shows the month-day portion of the relevant day; ID, which shows the guard on duty that day; and Minute, which shows the minutes during which the guard was asleep within the midnight hour. (The Minute column's header shows the minute's ten's digit in the first row and the one's digit in the second row.) Awake is shown as ., and asleep is shown as #.

Note that guards count as asleep on the minute they fall asleep, and they count as awake on the minute they wake up. For example, because Guard #10 wakes up at 00:25 on 1518-11-01, minute 25 is marked as awake.

If you can figure out the guard most likely to be asleep at a specific time, you might be able to trick that guard into working tonight so you can have the best chance of sneaking in. You have two strategies for choosing the best guard/minute combination.

Strategy 1: Find the guard that has the most minutes asleep. What minute does that guard spend asleep the most?

In the example above, Guard #10 spent the most minutes asleep, a total of 50 minutes (20+25+5), while Guard #99 only slept for a total of 30 minutes (10+10+10). Guard #10 was asleep most during minute 24 (on two days, whereas any other minute the guard was asleep was only seen on one day).

While this example listed the entries in chronological order, your entries are in the order you found them. You'll need to organize them before they can be analyzed.

What is the ID of the guard you chose multiplied by the minute you chose? (In the above example, the answer would be 10 * 24 = 240.)

In [17]:
function getSortedInput()
    inputs = Array{String}(undef, 0)
    open("day4input.txt") do f
        for line in eachline(f)
            push!(inputs, string(line))
        end
    end
    return sort(inputs)
end;
getSortedInput();

In [18]:
function extractMinute(line)
    return parse(Int64, line[16:17])
end

function extractGuardId(lines)
    split(lines[1])[4][2:end]
end

extractMinute("[1518-11-21 23:50] Guard #2953 begins shift")
# extractGuardId(["[1518-11-21 23:50] Guard #2953 begins shift"])

50

In [19]:
function partitionByGuard(sortedInput)
    guards = []
    guard = []
    for line in sortedInput
        if occursin("Guard", line) && length(guard) > 0
            push!(guards, guard)
            guard = []
        end
        push!(guard, line)
    end
    
    dict = Dict()
    for guard in guards
        id = extractGuardId(guard)
        dict[id] = append!(get(dict, id, []), guard)
    end
    return collect(values(dict))
end

partitionByGuard(getSortedInput())

23-element Array{Any,1}:
 Any["[1518-10-31 00:00] Guard #683 begins shift"]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
 Any["[1518-03-27 00:01] Guard #647 begins shift", "[1518-03-27 00:08] falls asleep", "[1518-03-27 00:12] wakes up", "[1518-03-27 00:22] falls asleep", "[1518-03-27 00:35] wakes up", "[1518-04-22 00:00] Guard #647 begins sh

In [20]:
function lineContainsSleepOrWakeTime(line)
    occursin(r"asleep|wakes", line)
end

lineContainsSleepOrWakeTime (generic function with 1 method)

In [21]:
function sumTimeAsleepForGuard(lines)
    id = extractGuardId(lines)
    sleepAndWakeMinutes = map(extractMinute, filter(lineContainsSleepOrWakeTime, lines))
    minutesAsleep = Iterators.partition(sleepAndWakeMinutes, 2) |>
        iter -> map(xy -> xy[2] - xy[1], iter) |>
        iter -> reduce(+, iter)
    return (id, minutesAsleep)
end

sumTimeAsleepForGuard(["[1518-11-01 00:00] Guard #10 begins shift";
"[1518-11-01 00:05] falls asleep";
"[1518-11-01 00:25] wakes up";
"[1518-11-01 00:30] falls asleep";
"[1518-11-01 00:55] wakes up"])

("10", 45)

In [22]:
function findGuardAsleepMost(inputs)
    guards = partitionByGuard(inputs)
    guardIdToTimeAsleep = map(sumTimeAsleepForGuard, guards)
    max = 0
    maxId = "N/A"
    for (id, timeAsleep) in guardIdToTimeAsleep
        if timeAsleep > max
            max = timeAsleep
            maxId = id
        end
    end
    return (maxId, max)
end
findGuardAsleepMost(getSortedInput())

("2953", 465)

In [23]:
function getGuardLinesForId(id, inputs)
    guards = partitionByGuard(inputs)
    for guard in guards
        if occursin("Guard #$id", guard[1])
            return guard
        end
    end
end
getGuardLinesForId("2953", getSortedInput())

67-element Array{Any,1}:
 "[1518-03-19 00:02] Guard #2953 begins shift"
 "[1518-03-19 00:37] falls asleep"            
 "[1518-03-19 00:41] wakes up"                
 "[1518-04-08 23:57] Guard #2953 begins shift"
 "[1518-04-09 00:31] falls asleep"            
 "[1518-04-09 00:47] wakes up"                
 "[1518-05-04 23:56] Guard #2953 begins shift"
 "[1518-05-05 00:07] falls asleep"            
 "[1518-05-05 00:17] wakes up"                
 "[1518-05-05 00:23] falls asleep"            
 "[1518-05-05 00:52] wakes up"                
 "[1518-05-08 23:59] Guard #2953 begins shift"
 "[1518-05-09 00:25] falls asleep"            
 ⋮                                            
 "[1518-10-07 00:55] wakes up"                
 "[1518-11-17 00:02] Guard #2953 begins shift"
 "[1518-11-17 00:22] falls asleep"            
 "[1518-11-17 00:41] wakes up"                
 "[1518-11-18 23:58] Guard #2953 begins shift"
 "[1518-11-19 00:12] falls asleep"            
 "[1518-11-19 00:59] wakes up"     

In [24]:
function findMinuteGuardIsAsleepMost(lines)
    sleepAndWakeMinutes = map(extractMinute, filter(lineContainsSleepOrWakeTime, lines))
    daysAsleepByMinute = Dict()
    for (asleep, awake) in Iterators.partition(sleepAndWakeMinutes, 2)
        for m in asleep:awake-1
            daysAsleepByMinute[m] = get(daysAsleepByMinute, m, 0) + 1
        end
    end
    max = 0
    maxMinute = -1
    for (minute, timesAsleep) in daysAsleepByMinute
        if timesAsleep > max
            max = timesAsleep
            maxMinute = minute
        end
    end
    return (max, maxMinute)
end

function solvePart1()
    inputs = getSortedInput()
    (id, minutes) = findGuardAsleepMost(inputs)
    guard = getGuardLinesForId(id, inputs)
    (max, minute) = findMinuteGuardIsAsleepMost(guard)
    return (id, minute)
end
solvePart1()

("2953", 39)

## Part Two

Strategy 2: Of all guards, which guard is most frequently asleep on the same minute?

In the example above, Guard #99 spent minute 45 asleep more than any other guard or minute - three times in total. (In all other cases, any guard spent any minute asleep at most twice.)

What is the ID of the guard you chose multiplied by the minute you chose? (In the above example, the answer would be 99 * 45 = 4455.)

In [25]:
function solvePart2()
    inputs = getSortedInput()
    guards = partitionByGuard(inputs)
    max = 0
    maxMinute = -1
    maxId = "n/a"
#     guardToMaxMinute = Dict()
#     for guard in guards
#         guardToMaxMinute[extractGuardId(guard)] = findMinuteGuardIsAsleepMost(guard)
#     end
#     return guardToMaxMinute
    for guard in guards
        (timesAsleep, minute) = findMinuteGuardIsAsleepMost(guard)
        if timesAsleep >= max
            max = timesAsleep
            maxMinute = minute
            maxId = extractGuardId(guard)
        end
    end
    return (maxId, maxMinute, max)
end
solvePart2()

("1069", 30, 17)

In [1]:
using DataStructures

In [2]:
function lettersReact(a, b)
    (a != b) && (lowercase(a) == lowercase(b))
end
lettersReact('c', 'C')

true

In [4]:
function reactString(s)
    q = deque(Char)
    for c in s
        if isempty(q)
            push!(q, c)
        else
            last = back(q)
            if lettersReact(c, last)
                pop!(q)
            else
                push!(q, c)
            end
        end
    end
    
    return join(q, "")
end
reactString("dabAcCaCBAcCcaDA")

"dabCBAcaDA"

In [19]:
function getInput()
    s = ""
    open("day5input.txt") do f
        s = read(f, String)
    end
    return s
end
length(reactString(getInput()))

9462

In [10]:
function getAllLowercaseLetters(s)
    letters = Set{Char}()
    for c in s
        push!(letters, lowercase(c))
    end
    return collect(letters)
end
getAllLowercaseLetters("aAbBcC")

3-element Array{Char,1}:
 'a'
 'c'
 'b'

In [166]:
function solveDay5Part2(s)
    minLen = typemax(Int32)
    cs = getAllLowercaseLetters(s)
    for c in cs
        alteredSequence = filter(c1 -> lowercase(c1) != lowercase(c), s)
        n = length(reactString(alteredSequence))
        minLen = min(n, minLen)
    end
    return minLen
end
solveDay5Part2(getInput())

UndefVarError: UndefVarError: getInput not defined

# Day 6: Chronal Coordinates

The device on your wrist beeps several times, and once again you feel like you're falling.

"Situation critical," the device announces. "Destination indeterminate. Chronal interference detected. Please specify new target coordinates."

The device then produces a list of coordinates (your puzzle input). Are they places it thinks are safe or dangerous? It recommends you check manual page 729. The Elves did not give you a manual.

If they're dangerous, maybe you can minimize the danger by finding the coordinate that gives the largest distance from the other points.

Using only the Manhattan distance, determine the area around each coordinate by counting the number of integer X,Y locations that are closest to that coordinate (and aren't tied in distance to any other coordinate).

Your goal is to find the size of the largest area that isn't infinite. For example, consider the following list of coordinates:

```
1, 1
1, 6
8, 3
3, 4
5, 5
8, 9
```

If we name these coordinates A through F, we can draw them on a grid, putting 0,0 at the top left:

```
..........
.A........
..........
........C.
...D......
.....E....
.B........
..........
..........
........F.
```

This view is partial - the actual grid extends infinitely in all directions. Using the Manhattan distance, each location's closest coordinate can be determined, shown here in lowercase:

```
aaaaa.cccc
aAaaa.cccc
aaaddecccc
aadddeccCc
..dDdeeccc
bb.deEeecc
bBb.eeee..
bbb.eeefff
bbb.eeffff
bbb.ffffFf
```
Locations shown as `.` are equally far from two or more coordinates, and so they don't count as being closest to any.

In this example, the areas of coordinates A, B, C, and F are infinite - while not shown here, their areas extend forever outside the visible grid. However, the areas of coordinates D and E are finite: D is closest to 9 locations, and E is closest to 17 (both including the coordinate's location itself). Therefore, in this example, the size of the largest area is 17.

What is the size of the largest area that isn't infinite?

In [199]:
using DelimitedFiles
function getDay6Input()
    open("day6input.txt") do f
        sorted = sortslices(readdlm(f, ',', Int64, '\n'), dims=1)
        mapslices(row -> (row[1], row[2]), sorted, dims=2)[:]
    end
end
day6Input = getDay6Input()

50-element Array{Tuple{Int64,Int64},1}:
 (42, 284) 
 (45, 299) 
 (57, 225) 
 (61, 100) 
 (63, 142) 
 (68, 124) 
 (70, 242) 
 (73, 266) 
 (75, 354) 
 (77, 98)  
 (77, 107) 
 (87, 306) 
 (92, 44)  
 ⋮         
 (306, 221)
 (312, 166)
 (318, 343)
 (322, 286)
 (327, 292)
 (329, 106)
 (330, 196)
 (342, 337)
 (344, 50) 
 (346, 273)
 (347, 125)
 (350, 173)

In [208]:
minimum(day6Input)

(350, 173)

In [219]:
function createGrid(input)
    xmin = minimum(map(first, input))
    ymin = minimum(map(tuple -> tuple[2], input))
    xmax = maximum(map(first, input))
    ymax = maximum(map(tuple -> tuple[2], input))
    xs = xmin-1:xmax+1
    ys = ymin-1:ymax+1
    squares = Base.product(xs, ys)
end
grid = collect(createGrid(day6Input))

311×313 Array{Tuple{Int64,Int64},2}:
 (41, 43)   (41, 44)   (41, 45)   …  (41, 353)   (41, 354)   (41, 355) 
 (42, 43)   (42, 44)   (42, 45)      (42, 353)   (42, 354)   (42, 355) 
 (43, 43)   (43, 44)   (43, 45)      (43, 353)   (43, 354)   (43, 355) 
 (44, 43)   (44, 44)   (44, 45)      (44, 353)   (44, 354)   (44, 355) 
 (45, 43)   (45, 44)   (45, 45)      (45, 353)   (45, 354)   (45, 355) 
 (46, 43)   (46, 44)   (46, 45)   …  (46, 353)   (46, 354)   (46, 355) 
 (47, 43)   (47, 44)   (47, 45)      (47, 353)   (47, 354)   (47, 355) 
 (48, 43)   (48, 44)   (48, 45)      (48, 353)   (48, 354)   (48, 355) 
 (49, 43)   (49, 44)   (49, 45)      (49, 353)   (49, 354)   (49, 355) 
 (50, 43)   (50, 44)   (50, 45)      (50, 353)   (50, 354)   (50, 355) 
 (51, 43)   (51, 44)   (51, 45)   …  (51, 353)   (51, 354)   (51, 355) 
 (52, 43)   (52, 44)   (52, 45)      (52, 353)   (52, 354)   (52, 355) 
 (53, 43)   (53, 44)   (53, 45)      (53, 353)   (53, 354)   (53, 355) 
 ⋮                         

In [246]:
function findDistances(grid, points)
    allDistances = Dict()
    for square in grid
        distances = map(point -> cityblock([x for x in point], [x for x in square]), points)
        allDistances[square] = distances
    end
    return allDistances
end
allDistancesMap = findDistances(grid, day6Input)

Dict{Any,Any} with 97343 entries:
  (192, 165) => [269, 281, 195, 196, 152, 165, 199, 220, 306, 182  …  304, 251,…
  (170, 123) => [289, 301, 215, 132, 126, 103, 219, 240, 326, 118  …  368, 315,…
  (130, 186) => [186, 198, 112, 155, 111, 124, 116, 137, 223, 141  …  345, 292,…
  (89, 353)  => [116, 98, 160, 281, 237, 250, 130, 103, 15, 267  …  239, 300, 2…
  (164, 109) => [297, 309, 223, 112, 134, 111, 227, 248, 334, 98  …  388, 335, …
  (47, 153)  => [136, 148, 82, 67, 27, 50, 112, 139, 229, 85  …  461, 408, 419,…
  (192, 223) => [211, 223, 137, 254, 210, 223, 141, 162, 248, 240  …  246, 193,…
  (132, 218) => [156, 168, 82, 189, 145, 158, 86, 107, 193, 175  …  311, 258, 2…
  (333, 68)  => [507, 519, 433, 304, 344, 321, 437, 458, 544, 286  …  290, 229,…
  (69, 239)  => [72, 84, 26, 147, 103, 116, 4, 31, 121, 149  …  353, 300, 311, …
  (110, 282) => [70, 82, 110, 231, 187, 200, 80, 53, 107, 217  …  269, 216, 227…
  (145, 103) => [284, 296, 210, 87, 121, 98, 214, 235, 321, 73  …  413, 360

In [239]:
function findClosestPoints(distancesMap, points)
    closest = Dict()
    for square in keys(distancesMap)
        distances = distancesMap[square]
        min_index = argmin(distances)
        min_point = points[min_index]
        min_distance = distances[min_index]
        if count(d -> d == min_distance, distances) == 1
            closest[square] = min_point
        else
            closest[square] = "tie"
        end
    end
    return closest
end

closestPointsMap = findClosestPoints(allDistancesMap, day6Input)

Dict{Any,Any} with 97343 entries:
  (192, 165) => (203, 127)
  (170, 123) => (150, 122)
  (130, 186) => (132, 194)
  (89, 353)  => (75, 354)
  (164, 109) => (184, 110)
  (47, 153)  => (63, 142)
  (192, 223) => (180, 220)
  (132, 218) => (111, 218)
  (333, 68)  => (344, 50)
  (69, 239)  => (70, 242)
  (110, 282) => (141, 283)
  (145, 103) => (142, 102)
  (148, 315) => (141, 317)
  (324, 158) => (312, 166)
  (301, 83)  => (329, 106)
  (309, 275) => (322, 286)
  (185, 289) => (190, 296)
  (256, 188) => (286, 206)
  (277, 127) => (347, 125)
  (116, 309) => (87, 306)
  (166, 345) => (141, 317)
  (89, 162)  => (103, 173)
  (202, 94)  => "tie"
  (282, 142) => (312, 166)
  (128, 253) => (141, 283)
  ⋮          => ⋮

In [240]:
function getSetOfPointsThatAreEligible(grid, points, closest)
    eligible = Set(points)
    
    (m, n) = size(grid)
    # Edges along the x direction
    for i in 1:m
        if closest[grid[i, 1]] in eligible
            pop!(eligible, closest[grid[i, 1]])
        end
        if closest[grid[i, n]] in eligible
            pop!(eligible, closest[grid[i, n]])
        end
    end
    # Edges along the y direction
    for i in 1:n
        if closest[grid[1, i]] in eligible
            pop!(eligible, closest[grid[1, i]])
        end
        if closest[grid[m, i]] in eligible
            pop!(eligible, closest[grid[m, i]])
        end
    end
    return eligible
end

eligiblePoints = getSetOfPointsThatAreEligible(grid, day6Input, closestPointsMap)

Set(Tuple{Int64,Int64}[(268, 245), (87, 306), (73, 266), (327, 292), (286, 206), (184, 238), (142, 102), (111, 218), (141, 283), (77, 98)  …  (135, 197), (103, 173), (184, 110), (203, 127), (180, 220), (77, 107), (150, 122), (131, 162), (152, 186), (322, 286), (144, 174)])

In [241]:
function findRegionSizes(closest, eligible)
    regionSizes = Dict()
    for square in keys(closest)
        p = closest[square]
        if p in eligible
            regionSizes[p] = get(regionSizes, p, 0) + 1
        end
    end
    return regionSizes
end

regionSizes = findRegionSizes(closestPointsMap, eligiblePoints)

Dict{Any,Any} with 24 entries:
  (77, 107)  => 539
  (135, 197) => 603
  (111, 218) => 2242
  (141, 283) => 2741
  (103, 173) => 2200
  (77, 98)   => 1090
  (268, 245) => 4346
  (87, 306)  => 2194
  (150, 122) => 1342
  (184, 110) => 1772
  (73, 266)  => 1637
  (131, 162) => 1077
  (203, 127) => 5429
  (127, 107) => 1390
  (312, 166) => 3688
  (327, 292) => 1395
  (132, 194) => 356
  (180, 220) => 2660
  (286, 206) => 3181
  (152, 186) => 1325
  (322, 286) => 1582
  (184, 238) => 2653
  (142, 102) => 1236
  (144, 174) => 1230

In [242]:
function findMaxSize(regionSizes)
    maximum(values(regionSizes))
end
findMaxSize(regionSizes)

5429

## Part Two

On the other hand, if the coordinates are safe, maybe the best you can do is try to find a region near as many coordinates as possible.

For example, suppose you want the sum of the Manhattan distance to all of the coordinates to be less than 32. For each location, add up the distances to all of the given coordinates; if the total of those distances is less than 32, that location is within the desired region. Using the same coordinates as above, the resulting region looks like this:

```
..........
.A........
..........
...###..C.
..#D###...
..###E#...
.B.###....
..........
..........
........F.
```
In particular, consider the highlighted location 4,3 located at the top middle of the region. Its calculation is as follows, where abs() is the absolute value function:

Distance to coordinate A: `abs(4-1) + abs(3-1) =  5`

Distance to coordinate B: `abs(4-1) + abs(3-6) =  6`

Distance to coordinate C: `abs(4-8) + abs(3-3) =  4`

Distance to coordinate D: `abs(4-3) + abs(3-4) =  2`

Distance to coordinate E: `abs(4-5) + abs(3-5) =  3`

Distance to coordinate F: `abs(4-8) + abs(3-9) = 10`

Total distance: 5 + 6 + 4 + 2 + 3 + 10 = 30

Because the total distance to all coordinates (30) is less than 32, the location is within the region.

This region, which also includes coordinates D and E, has a total size of 16.

Your actual region will need to be much larger than this example, though, instead including all locations with a total distance of less than 10000.

What is the size of the region containing all locations which have a total distance to all given coordinates of less than 10000?

In [259]:
function solvePart2(allDistances)
    totalDistances = Dict()
    for (square, distances) in allDistancesMap
        totalDistances[square] = sum(distances)
    end
    return totalDistances
end
filter(d -> d < 10000, collect(values(solvePart2(allDistancesMap))))

32614-element Array{Any,1}:
 8767
 9381
 8601
 9733
 8659
 8601
 9909
 9777
 9451
 9691
 9127
 9267
 9573
    ⋮
 9873
 9591
 9811
 8507
 8621
 9603
 9093
 9945
 9145
 9651
 9605
 9043

# Day 7: The Sum of Its Parts

You find yourself standing on a snow-covered coastline; apparently, you landed a little off course. The region is too hilly to see the North Pole from here, but you do spot some Elves that seem to be trying to unpack something that washed ashore. It's quite cold out, so you decide to risk creating a paradox by asking them for directions.

"Oh, are you the search party?" Somehow, you can understand whatever Elves from the year 1018 speak; you assume it's Ancient Nordic Elvish. Could the device on your wrist also be a translator? "Those clothes don't look very warm; take this." They hand you a heavy coat.

"We do need to find our way back to the North Pole, but we have higher priorities at the moment. You see, believe it or not, this box contains something that will solve all of Santa's transportation problems - at least, that's what it looks like from the pictures in the instructions." It doesn't seem like they can read whatever language it's in, but you can: "Sleigh kit. Some assembly required."

"'Sleigh'? What a wonderful name! You must help us assemble this 'sleigh' at once!" They start excitedly pulling more parts out of the box.

The instructions specify a series of steps and requirements about which steps must be finished before others can begin (your puzzle input). Each step is designated by a single letter. For example, suppose you have the following instructions:

Step C must be finished before step A can begin.

Step C must be finished before step F can begin.

Step A must be finished before step B can begin.

Step A must be finished before step D can begin.

Step B must be finished before step E can begin.

Step D must be finished before step E can begin.

Step F must be finished before step E can begin.

Visually, these requirements look like this:


```
  -->A--->B--
 /    \      \
C      -->D----->E
 \           /
  ---->F-----
```

Your first goal is to determine the order in which the steps should be completed. If more than one step is ready, choose the step which is first alphabetically. In this example, the steps would be completed as follows:

Only C is available, and so it is done first.
Next, both A and F are available. A is first alphabetically, so it is done next.
Then, even though F was available earlier, steps B and D are now also available, and B is the first alphabetically of the three.
After that, only D and F are available. E is not available because only some of its prerequisites are complete. Therefore, D is completed next.
F is the only choice, so it is done next.
Finally, E is completed.
So, in this example, the correct order is CABDFE.

In what order should the steps in your instructions be completed?

In [305]:
function getDay7Lines()
    lines = []
    open("day7input.txt") do f
        for line in eachline(f)
            push!(lines, line)
        end
    end
    return lines
end

function parseInput(lines)
    parents = Dict()
    children = Dict()
    for s in lines
        step, required = s[37], s[6]
        parents[step] = union(get(parents, step, Set()), Set([required]))
        children[required] = union(get(children, required, Set()), Set([step]))
    end
    
    return (parents, children) 
end
parents, children = parseInput(getDay7Lines());

(Dict{Any,Any}('E'=>Set(Any['P', 'J', 'Y', 'R', 'F', 'O']),'Z'=>Set(Any['M', 'S']),'X'=>Set(Any['G', 'N', 'Q']),'C'=>Set(Any['J', 'R', 'S']),'B'=>Set(Any['P', 'E', 'Z', 'V', 'T']),'D'=>Set(Any['E', 'B', 'A', 'K', 'J', 'I', 'H', 'S', 'U', 'L', 'Y']),'A'=>Set(Any['P', 'K', 'L', 'Y', 'E', 'G', 'B']),'R'=>Set(Any['M']),'G'=>Set(Any['W']),'O'=>Set(Any['M', 'Z', 'X'])…), Dict{Any,Any}('E'=>Set(Any['U', 'D', 'A', 'L', 'I', 'H', 'B']),'Z'=>Set(Any['J', 'Y', 'O', 'B']),'X'=>Set(Any['P', 'I', 'L', 'H', 'O']),'C'=>Set(Any['I', 'Y', 'T']),'B'=>Set(Any['U', 'D', 'A', 'I', 'H']),'A'=>Set(Any['U', 'D', 'I', 'H']),'R'=>Set(Any['P', 'I', 'Y', 'E', 'C']),'G'=>Set(Any['U', 'J', 'A', 'L', 'I', 'H', 'X']),'Q'=>Set(Any['P', 'K', 'U', 'X']),'N'=>Set(Any['K', 'X'])…))

(Dict{Any,Any}('D'=>Set(Any['A']),'A'=>Set(Any['C']),'E'=>Set(Any['D', 'F', 'B']),'F'=>Set(Any['C']),'B'=>Set(Any['A'])), Dict{Any,Any}('B'=>Set(Any['E']),'D'=>Set(Any['E']),'A'=>Set(Any['D', 'B']),'F'=>Set(Any['E']),'C'=>Set(Any['A', 'F'])))

In [314]:
function solvePart1(parents, children)
    queue = PriorityQueue{Char, Char}()
    roots = setdiff(Set(keys(children)), Set(keys(parents)))
    for root in roots
        enqueue!(queue, root, root)
    end
    
    order = []
    processed = Set()
    while length(queue) > 0
        c = dequeue!(queue)
        push!(processed, c)
        push!(order, c)
        
        childs = get(children, c, Set())
        for child in childs
            deps = get(parents, child, Set())
            if length(setdiff(parents[child], processed)) == 0
                enqueue!(queue, child, child)
            end
        end
    end
    join(order, "")
end

solvePart1(parents, children)

"MNQKRSFWGXPZJCOTVYEBLAHIUD"

## Part Two

As you're about to begin construction, four of the Elves offer to help. "The sun will set soon; it'll go faster if we work together." Now, you need to account for multiple people working on steps simultaneously. If multiple steps are available, workers should still begin them in alphabetical order.

Each step takes 60 seconds plus an amount corresponding to its letter: A=1, B=2, C=3, and so on. So, step A takes 60+1=61 seconds, while step Z takes 60+26=86 seconds. No time is required between steps.

To simplify things for the example, however, suppose you only have help from one Elf (a total of two workers) and that each step takes 60 fewer seconds (so that step A takes 1 second and step Z takes 26 seconds). Then, using the same instructions as above, this is how each second would be spent:

```
Second   Worker 1   Worker 2   Done
   0        C          .        
   1        C          .        
   2        C          .        
   3        A          F       C
   4        B          F       CA
   5        B          F       CA
   6        D          F       CAB
   7        D          F       CAB
   8        D          F       CAB
   9        D          .       CABF
  10        E          .       CABFD
  11        E          .       CABFD
  12        E          .       CABFD
  13        E          .       CABFD
  14        E          .       CABFD
  15        .          .       CABFDE
```

Each row represents one second of time. The Second column identifies how many seconds have passed as of the beginning of that second. Each worker column shows the step that worker is currently doing (or . if they are idle). The Done column shows completed steps.

Note that the order of the steps has changed; this is because steps now take time to finish and multiple workers can begin multiple steps simultaneously.

In this example, it would take 15 seconds for two workers to complete these steps.

With 5 workers and the 60+ second step durations described above, how long will it take to complete all of the steps?

In [317]:
function timeToDo(task)
    60 + task - 'A' + 1
end

function addToDictionary!(d, time, task)
    d[time] = [get(d, time, []); task]
end

function enqueueUnblockedChildren!(q, c, children, parents, processed)
    childs = get(children, c, Set())
    for child in childs
        deps = get(parents, child, Set())
        if length(setdiff(parents[child], processed)) == 0
            enqueue!(q, child, child)
        end
    end
end

function solvePart2(parents, children, freeWorkers)
    queue = PriorityQueue{Char, Char}()
    roots = setdiff(Set(keys(children)), Set(keys(parents)))
    for root in roots
        enqueue!(queue, root, root)
    end

    processed = Set()
    seconds = 0
    # a map with 15 -> ['A', 'B'], where 'A' and 'B' are tasks, and 15 is the
    # seconds count when they will finish.
    inProgress = Dict()
    while (length(queue) + length(keys(inProgress)) > 0)
        # Remove completed tasks
        if seconds in keys(inProgress)
            for completed in inProgress[seconds]
                push!(processed, completed)
                enqueueUnblockedChildren!(queue, completed, children, parents, processed)
                freeWorkers += 1
            end
            delete!(inProgress, seconds)
        end
        # To avoid an off-by-one error, break immediately when we finish
        if length(queue) == 0 && length(keys(inProgress)) == 0
            break
        end
        
        # Start as many workers as we have and have tasks for
        while freeWorkers > 0 && length(queue) > 0
            c = dequeue!(queue)
            addToDictionary!(inProgress, seconds + timeToDo(c), c)
            freeWorkers -= 1
        end
        seconds += 1
        println(inProgress, " ", processed, " ", seconds, " ", freeWorkers)
    end
    return seconds
end

solvePart2(parents, children, 5)

Dict{Any,Any}(77=>Any['Q'],83=>Any['W'],74=>Any['N'],73=>Any['M']) Set(Any[]) 1 1
Dict{Any,Any}(77=>Any['Q'],83=>Any['W'],74=>Any['N'],73=>Any['M']) Set(Any[]) 2 1
Dict{Any,Any}(77=>Any['Q'],83=>Any['W'],74=>Any['N'],73=>Any['M']) Set(Any[]) 3 1
Dict{Any,Any}(77=>Any['Q'],83=>Any['W'],74=>Any['N'],73=>Any['M']) Set(Any[]) 4 1
Dict{Any,Any}(77=>Any['Q'],83=>Any['W'],74=>Any['N'],73=>Any['M']) Set(Any[]) 5 1
Dict{Any,Any}(77=>Any['Q'],83=>Any['W'],74=>Any['N'],73=>Any['M']) Set(Any[]) 6 1
Dict{Any,Any}(77=>Any['Q'],83=>Any['W'],74=>Any['N'],73=>Any['M']) Set(Any[]) 7 1
Dict{Any,Any}(77=>Any['Q'],83=>Any['W'],74=>Any['N'],73=>Any['M']) Set(Any[]) 8 1
Dict{Any,Any}(77=>Any['Q'],83=>Any['W'],74=>Any['N'],73=>Any['M']) Set(Any[]) 9 1
Dict{Any,Any}(77=>Any['Q'],83=>Any['W'],74=>Any['N'],73=>Any['M']) Set(Any[]) 10 1
Dict{Any,Any}(77=>Any['Q'],83=>Any['W'],74=>Any['N'],73=>Any['M']) Set(Any[]) 11 1
Dict{Any,Any}(77=>Any['Q'],83=>Any['W'],74=>Any['N'],73=>Any['M']) Set(Any[]) 12 1
Dict{Any,Any}

Dict{Any,Any}(148=>Any['K'],151=>Any['R'],152=>Any['S'],150=>Any['G']) Set(Any['M', 'W', 'N', 'Q']) 114 1
Dict{Any,Any}(148=>Any['K'],151=>Any['R'],152=>Any['S'],150=>Any['G']) Set(Any['M', 'W', 'N', 'Q']) 115 1
Dict{Any,Any}(148=>Any['K'],151=>Any['R'],152=>Any['S'],150=>Any['G']) Set(Any['M', 'W', 'N', 'Q']) 116 1
Dict{Any,Any}(148=>Any['K'],151=>Any['R'],152=>Any['S'],150=>Any['G']) Set(Any['M', 'W', 'N', 'Q']) 117 1
Dict{Any,Any}(148=>Any['K'],151=>Any['R'],152=>Any['S'],150=>Any['G']) Set(Any['M', 'W', 'N', 'Q']) 118 1
Dict{Any,Any}(148=>Any['K'],151=>Any['R'],152=>Any['S'],150=>Any['G']) Set(Any['M', 'W', 'N', 'Q']) 119 1
Dict{Any,Any}(148=>Any['K'],151=>Any['R'],152=>Any['S'],150=>Any['G']) Set(Any['M', 'W', 'N', 'Q']) 120 1
Dict{Any,Any}(148=>Any['K'],151=>Any['R'],152=>Any['S'],150=>Any['G']) Set(Any['M', 'W', 'N', 'Q']) 121 1
Dict{Any,Any}(148=>Any['K'],151=>Any['R'],152=>Any['S'],150=>Any['G']) Set(Any['M', 'W', 'N', 'Q']) 122 1
Dict{Any,Any}(148=>Any['K'],151=>Any['R'],152=

Dict{Any,Any}(234=>Any['X'],238=>Any['Z'],218=>Any['F']) Set(Any['M', 'K', 'S', 'R', 'W', 'G', 'N', 'Q']) 214 2
Dict{Any,Any}(234=>Any['X'],238=>Any['Z'],218=>Any['F']) Set(Any['M', 'K', 'S', 'R', 'W', 'G', 'N', 'Q']) 215 2
Dict{Any,Any}(234=>Any['X'],238=>Any['Z'],218=>Any['F']) Set(Any['M', 'K', 'S', 'R', 'W', 'G', 'N', 'Q']) 216 2
Dict{Any,Any}(234=>Any['X'],238=>Any['Z'],218=>Any['F']) Set(Any['M', 'K', 'S', 'R', 'W', 'G', 'N', 'Q']) 217 2
Dict{Any,Any}(234=>Any['X'],238=>Any['Z'],218=>Any['F']) Set(Any['M', 'K', 'S', 'R', 'W', 'G', 'N', 'Q']) 218 2
Dict{Any,Any}(234=>Any['X'],238=>Any['Z']) Set(Any['M', 'K', 'S', 'F', 'R', 'W', 'G', 'N', 'Q']) 219 3
Dict{Any,Any}(234=>Any['X'],238=>Any['Z']) Set(Any['M', 'K', 'S', 'F', 'R', 'W', 'G', 'N', 'Q']) 220 3
Dict{Any,Any}(234=>Any['X'],238=>Any['Z']) Set(Any['M', 'K', 'S', 'F', 'R', 'W', 'G', 'N', 'Q']) 221 3
Dict{Any,Any}(234=>Any['X'],238=>Any['Z']) Set(Any['M', 'K', 'S', 'F', 'R', 'W', 'G', 'N', 'Q']) 222 3
Dict{Any,Any}(234=>Any['X'],

Dict{Any,Any}(310=>Any['P'],308=>Any['J'],313=>Any['O']) Set(Any['Z', 'X', 'R', 'G', 'F', 'N', 'M', 'K', 'Q', 'W', 'S']) 304 2
Dict{Any,Any}(310=>Any['P'],308=>Any['J'],313=>Any['O']) Set(Any['Z', 'X', 'R', 'G', 'F', 'N', 'M', 'K', 'Q', 'W', 'S']) 305 2
Dict{Any,Any}(310=>Any['P'],308=>Any['J'],313=>Any['O']) Set(Any['Z', 'X', 'R', 'G', 'F', 'N', 'M', 'K', 'Q', 'W', 'S']) 306 2
Dict{Any,Any}(310=>Any['P'],308=>Any['J'],313=>Any['O']) Set(Any['Z', 'X', 'R', 'G', 'F', 'N', 'M', 'K', 'Q', 'W', 'S']) 307 2
Dict{Any,Any}(310=>Any['P'],308=>Any['J'],313=>Any['O']) Set(Any['Z', 'X', 'R', 'G', 'F', 'N', 'M', 'K', 'Q', 'W', 'S']) 308 2
Dict{Any,Any}(310=>Any['P'],371=>Any['C'],313=>Any['O']) Set(Any['Z', 'X', 'R', 'G', 'F', 'N', 'M', 'K', 'Q', 'J', 'W', 'S']) 309 2
Dict{Any,Any}(310=>Any['P'],371=>Any['C'],313=>Any['O']) Set(Any['Z', 'X', 'R', 'G', 'F', 'N', 'M', 'K', 'Q', 'J', 'W', 'S']) 310 2
Dict{Any,Any}(371=>Any['C'],313=>Any['O'],392=>Any['V']) Set(Any['Z', 'X', 'R', 'G', 'F', 'N', 'M', '

Dict{Any,Any}(451=>Any['T'],392=>Any['V']) Set(Any['Z', 'X', 'C', 'R', 'G', 'F', 'N', 'M', 'K', 'Q', 'J', 'P', 'O', 'W', 'S']) 383 3
Dict{Any,Any}(451=>Any['T'],392=>Any['V']) Set(Any['Z', 'X', 'C', 'R', 'G', 'F', 'N', 'M', 'K', 'Q', 'J', 'P', 'O', 'W', 'S']) 384 3
Dict{Any,Any}(451=>Any['T'],392=>Any['V']) Set(Any['Z', 'X', 'C', 'R', 'G', 'F', 'N', 'M', 'K', 'Q', 'J', 'P', 'O', 'W', 'S']) 385 3
Dict{Any,Any}(451=>Any['T'],392=>Any['V']) Set(Any['Z', 'X', 'C', 'R', 'G', 'F', 'N', 'M', 'K', 'Q', 'J', 'P', 'O', 'W', 'S']) 386 3
Dict{Any,Any}(451=>Any['T'],392=>Any['V']) Set(Any['Z', 'X', 'C', 'R', 'G', 'F', 'N', 'M', 'K', 'Q', 'J', 'P', 'O', 'W', 'S']) 387 3
Dict{Any,Any}(451=>Any['T'],392=>Any['V']) Set(Any['Z', 'X', 'C', 'R', 'G', 'F', 'N', 'M', 'K', 'Q', 'J', 'P', 'O', 'W', 'S']) 388 3
Dict{Any,Any}(451=>Any['T'],392=>Any['V']) Set(Any['Z', 'X', 'C', 'R', 'G', 'F', 'N', 'M', 'K', 'Q', 'J', 'P', 'O', 'W', 'S']) 389 3
Dict{Any,Any}(451=>Any['T'],392=>Any['V']) Set(Any['Z', 'X', 'C', 'R'

...

Dict{Any,Any}(803=>Any['I'],802=>Any['H']) Set(Any['E', 'Z', 'X', 'C', 'B', 'A', 'R', 'G', 'F', 'N', 'M', 'K', 'Q', 'J', 'P', 'O', 'W', 'S', 'T', 'L', 'Y', 'V']) 737 3
Dict{Any,Any}(803=>Any['I'],802=>Any['H']) Set(Any['E', 'Z', 'X', 'C', 'B', 'A', 'R', 'G', 'F', 'N', 'M', 'K', 'Q', 'J', 'P', 'O', 'W', 'S', 'T', 'L', 'Y', 'V']) 738 3
Dict{Any,Any}(803=>Any['I'],802=>Any['H']) Set(Any['E', 'Z', 'X', 'C', 'B', 'A', 'R', 'G', 'F', 'N', 'M', 'K', 'Q', 'J', 'P', 'O', 'W', 'S', 'T', 'L', 'Y', 'V']) 739 3
Dict{Any,Any}(803=>Any['I'],802=>Any['H']) Set(Any['E', 'Z', 'X', 'C', 'B', 'A', 'R', 'G', 'F', 'N', 'M', 'K', 'Q', 'J', 'P', 'O', 'W', 'S', 'T', 'L', 'Y', 'V']) 740 3
Dict{Any,Any}(803=>Any['I'],802=>Any['H']) Set(Any['E', 'Z', 'X', 'C', 'B', 'A', 'R', 'G', 'F', 'N', 'M', 'K', 'Q', 'J', 'P', 'O', 'W', 'S', 'T', 'L', 'Y', 'V']) 741 3
Dict{Any,Any}(803=>Any['I'],802=>Any['H']) Set(Any['E', 'Z', 'X', 'C', 'B', 'A', 'R', 'G', 'F', 'N', 'M', 'K', 'Q', 'J', 'P', 'O', 'W', 'S', 'T', 'L', 'Y', 'V']

Dict{Any,Any}(803=>Any['I'],802=>Any['H']) Set(Any['E', 'Z', 'X', 'C', 'B', 'A', 'R', 'G', 'F', 'N', 'M', 'K', 'Q', 'J', 'P', 'O', 'W', 'S', 'T', 'L', 'Y', 'V']) 790 3
Dict{Any,Any}(803=>Any['I'],802=>Any['H']) Set(Any['E', 'Z', 'X', 'C', 'B', 'A', 'R', 'G', 'F', 'N', 'M', 'K', 'Q', 'J', 'P', 'O', 'W', 'S', 'T', 'L', 'Y', 'V']) 791 3
Dict{Any,Any}(803=>Any['I'],802=>Any['H']) Set(Any['E', 'Z', 'X', 'C', 'B', 'A', 'R', 'G', 'F', 'N', 'M', 'K', 'Q', 'J', 'P', 'O', 'W', 'S', 'T', 'L', 'Y', 'V']) 792 3
Dict{Any,Any}(803=>Any['I'],802=>Any['H']) Set(Any['E', 'Z', 'X', 'C', 'B', 'A', 'R', 'G', 'F', 'N', 'M', 'K', 'Q', 'J', 'P', 'O', 'W', 'S', 'T', 'L', 'Y', 'V']) 793 3
Dict{Any,Any}(803=>Any['I'],802=>Any['H']) Set(Any['E', 'Z', 'X', 'C', 'B', 'A', 'R', 'G', 'F', 'N', 'M', 'K', 'Q', 'J', 'P', 'O', 'W', 'S', 'T', 'L', 'Y', 'V']) 794 3
Dict{Any,Any}(803=>Any['I'],802=>Any['H']) Set(Any['E', 'Z', 'X', 'C', 'B', 'A', 'R', 'G', 'F', 'N', 'M', 'K', 'Q', 'J', 'P', 'O', 'W', 'S', 'T', 'L', 'Y', 'V']

Dict{Any,Any}(884=>Any['U']) Set(Any['E', 'Z', 'X', 'C', 'B', 'A', 'R', 'G', 'F', 'N', 'M', 'K', 'Q', 'J', 'P', 'O', 'H', 'I', 'W', 'S', 'T', 'L', 'Y', 'V']) 848 4
Dict{Any,Any}(884=>Any['U']) Set(Any['E', 'Z', 'X', 'C', 'B', 'A', 'R', 'G', 'F', 'N', 'M', 'K', 'Q', 'J', 'P', 'O', 'H', 'I', 'W', 'S', 'T', 'L', 'Y', 'V']) 849 4
Dict{Any,Any}(884=>Any['U']) Set(Any['E', 'Z', 'X', 'C', 'B', 'A', 'R', 'G', 'F', 'N', 'M', 'K', 'Q', 'J', 'P', 'O', 'H', 'I', 'W', 'S', 'T', 'L', 'Y', 'V']) 850 4
Dict{Any,Any}(884=>Any['U']) Set(Any['E', 'Z', 'X', 'C', 'B', 'A', 'R', 'G', 'F', 'N', 'M', 'K', 'Q', 'J', 'P', 'O', 'H', 'I', 'W', 'S', 'T', 'L', 'Y', 'V']) 851 4
Dict{Any,Any}(884=>Any['U']) Set(Any['E', 'Z', 'X', 'C', 'B', 'A', 'R', 'G', 'F', 'N', 'M', 'K', 'Q', 'J', 'P', 'O', 'H', 'I', 'W', 'S', 'T', 'L', 'Y', 'V']) 852 4
Dict{Any,Any}(884=>Any['U']) Set(Any['E', 'Z', 'X', 'C', 'B', 'A', 'R', 'G', 'F', 'N', 'M', 'K', 'Q', 'J', 'P', 'O', 'H', 'I', 'W', 'S', 'T', 'L', 'Y', 'V']) 853 4
Dict{Any,Any}(88

Dict{Any,Any}(948=>Any['D']) Set(Any['E', 'Z', 'X', 'C', 'B', 'A', 'R', 'G', 'F', 'N', 'M', 'K', 'Q', 'J', 'P', 'O', 'H', 'I', 'W', 'S', 'T', 'U', 'L', 'Y', 'V']) 907 4
Dict{Any,Any}(948=>Any['D']) Set(Any['E', 'Z', 'X', 'C', 'B', 'A', 'R', 'G', 'F', 'N', 'M', 'K', 'Q', 'J', 'P', 'O', 'H', 'I', 'W', 'S', 'T', 'U', 'L', 'Y', 'V']) 908 4
Dict{Any,Any}(948=>Any['D']) Set(Any['E', 'Z', 'X', 'C', 'B', 'A', 'R', 'G', 'F', 'N', 'M', 'K', 'Q', 'J', 'P', 'O', 'H', 'I', 'W', 'S', 'T', 'U', 'L', 'Y', 'V']) 909 4
Dict{Any,Any}(948=>Any['D']) Set(Any['E', 'Z', 'X', 'C', 'B', 'A', 'R', 'G', 'F', 'N', 'M', 'K', 'Q', 'J', 'P', 'O', 'H', 'I', 'W', 'S', 'T', 'U', 'L', 'Y', 'V']) 910 4
Dict{Any,Any}(948=>Any['D']) Set(Any['E', 'Z', 'X', 'C', 'B', 'A', 'R', 'G', 'F', 'N', 'M', 'K', 'Q', 'J', 'P', 'O', 'H', 'I', 'W', 'S', 'T', 'U', 'L', 'Y', 'V']) 911 4
Dict{Any,Any}(948=>Any['D']) Set(Any['E', 'Z', 'X', 'C', 'B', 'A', 'R', 'G', 'F', 'N', 'M', 'K', 'Q', 'J', 'P', 'O', 'H', 'I', 'W', 'S', 'T', 'U', 'L', 'Y'

948

In [308]:
sampleParents, sampleChildren = parseInput(["Step C must be finished before step A can begin.",
"Step C must be finished before step F can begin.",
"Step A must be finished before step B can begin.",
"Step A must be finished before step D can begin.",
"Step B must be finished before step E can begin.",
"Step D must be finished before step E can begin.",
"Step F must be finished before step E can begin."])
solvePart2(sampleParents, sampleChildren, 2)

63